In [1]:
import numpy as np
from typing import Callable, Dict, List
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Начальное состояние
x0, y0 = 100, 200_000
K_squared = x0 * y0

# Swap с комиссией
dx = 50
gamma = 0.997

x_new = x0 + gamma * dx
y_new = K_squared / x_new
dy = y0 - y_new

# Финальное состояние (с реинвестированием комиссии)
x_final = x0 + dx
y_final = y0 - dy

print(f"НАЧАЛО:")
print(f"  Резервы: {x0} ETH, {y0:,} USDC")
print(f"  Цена: {y0/x0:,.1f} USDC/ETH")
print()

print(f"ОБМЕН:")
print(f"  Отдал: {dx} ETH")
print(f"  Получил: {dy:,.1f} USDC")
print(f"  Эффективная цена: {dy/dx:,.1f} USDC/ETH")
print()

print(f"КОНЕЦ:")
print(f"  Резервы: {x_final:.1f} ETH, {y_final:,.1f} USDC")
print(f"  Цена: {y_final/x_final:,.1f} USDC/ETH")
print()

fig = go.Figure()

# Кривая инварианта ДО обмена
x_range = np.linspace(50, 200, 500)
y_curve_before = K_squared / x_range
fig.add_trace(go.Scatter(
    x=x_range, y=y_curve_before,
    mode='lines',
    line=dict(color='blue', width=3),
    name=f'До обмена: x·y = {K_squared:,}'
))

# Кривая инварианта ПОСЛЕ обмена
K_squared_after = x_final * y_final
y_curve_after = K_squared_after / x_range
fig.add_trace(go.Scatter(
    x=x_range, y=y_curve_after,
    mode='lines',
    line=dict(color='green', width=3, dash='dash'),
    name=f'После обмена: x·y = {K_squared_after:,.0f}'
))

# Начальная точка
fig.add_trace(go.Scatter(
    x=[x0], y=[y0],
    mode='markers+text',
    marker=dict(color='blue', size=15),
    text=['НАЧАЛО'],
    textposition='top center',
    name=f'Начало ({x0}, {y0:,})'
))

# Конечная точка
fig.add_trace(go.Scatter(
    x=[x_final], y=[y_final],
    mode='markers+text',
    marker=dict(color='red', size=15),
    text=['КОНЕЦ'],
    textposition='bottom center',
    name=f'Конец ({x_final:.0f}, {y_final:,.0f})'
))

# Линия обмена
fig.add_trace(go.Scatter(
    x=[x0, x_final],
    y=[y0, y_final],
    mode='lines',
    line=dict(color='red', width=2, dash='dot'),
    name='Траектория обмена'
))

# Касательная в начальной точке (мгновенная цена)
price_instant = y0 / x0
x_tangent = np.linspace(80, 120, 100)
y_tangent = y0 - price_instant * (x_tangent - x0)
fig.add_trace(go.Scatter(
    x=x_tangent, y=y_tangent,
    mode='lines',
    line=dict(color='purple', width=2, dash='dash'),
    name=f'Мгновенная цена: {price_instant:,.0f} USDC/ETH'
))

# Линия эффективной цены
effective_price = dy / dx
y_effective = y0 - effective_price * (x_range - x0)
fig.add_trace(go.Scatter(
    x=x_range, y=y_effective,
    mode='lines',
    line=dict(color='orange', width=2, dash='dot'),
    name=f'Эффективная цена: {effective_price:,.1f} USDC/ETH'
))

# Касательная в конечной точке (цена после)
price_after = y_final / x_final
x_tangent_after = np.linspace(130, 170, 100)
y_tangent_after = y_final - price_after * (x_tangent_after - x_final)
fig.add_trace(go.Scatter(
    x=x_tangent_after, y=y_tangent_after,
    mode='lines',
    line=dict(color='green', width=2, dash='dash'),
    name=f'Цена после: {price_after:,.1f} USDC/ETH'
))

fig.update_layout(
    title='Три разные цены в AMM',
    xaxis_title='Резерв X (ETH)',
    yaxis_title='Резерв Y (USDC)',
    hovermode='closest',
    width=1000,
    height=700
)

fig.show()

НАЧАЛО:
  Резервы: 100 ETH, 200,000 USDC
  Цена: 2,000.0 USDC/ETH

ОБМЕН:
  Отдал: 50 ETH
  Получил: 66,533.2 USDC
  Эффективная цена: 1,330.7 USDC/ETH

КОНЕЦ:
  Резервы: 150.0 ETH, 133,466.8 USDC
  Цена: 889.8 USDC/ETH



In [3]:
class UniswapPool:
    """
    Uniswap v2 with Dynamic Fees Simulator and Impermanent Loss Analysis
    
    The main formula: x · y = K
    With fee: (x + γ·Δx) · (y - Δy) = K
    where γ = 1 - φ(Δx) - dynamic fee function
    """
    
    def __init__(self, x0: float, y0: float, fee_function: Callable = None):
        """
        Args:
            x0: base token of X
            y0: base token of Y
            fee_function: Функция комиссии φ(dx, x, y)
                         Если None, используется фиксированная 0.3%
        """
        self.x0 = x0
        self.y0 = y0
        self.x = x0
        self.y = y0
        self.K = x0 * y0
        
        if fee_function is None:
            self.fee_function = lambda dx, x, y: 0.003
        else:
            self.fee_function = fee_function
    
    def reset(self):
        """Сброс пула в начальное состояние"""
        self.x = self.x0
        self.y = self.y0
        self.K = self.x0 * self.y0
    
    def get_price(self) -> float:
        """Получить текущую мгновенную цену (сколько Y за 1 X)"""
        return self.y / self.x
    
    def get_current_state(self) -> Dict:
        """Вернуть текущее состояние пула"""
        return {
            'x': self.x,
            'y': self.y,
            'price': self.get_price(),
            'K': self.K
        }
    
    def swap(self, dx: float) -> Dict:
        """
        Выполнить обмен dx токенов X на dy токенов Y
        
        Args:
            dx: Количество токенов X для обмена
            
        Returns:
            Dict с информацией об обмене
        """
        # Цена ДО обмена
        price_before = self.get_price()
        K_before = self.K
        
        # Вычисляем комиссию для ЭТОГО обмена
        fee_rate = self.fee_function(dx, self.x, self.y)
        gamma = 1 - fee_rate
        
        # Обмен с учетом комиссии
        # (x + γ·dx) · (y - dy) = K²
        x_new = self.x + gamma * dx
        y_new = self.K / x_new
        dy = self.y - y_new
        
        # Обновляем резервы (комиссия реинвестируется!)
        self.x += dx  # Весь dx остается в пуле
        self.y -= dy  # Убираем только dy
        
        # Обновляем K (растет из-за комиссий)
        self.K = self.x * self.y
        K_after = self.K
        
        # Цена ПОСЛЕ обмена
        price_after = self.get_price()
        
        # Проскальзывание
        slippage = (price_before - dy/dx) / price_before * 100
        
        # ════════════════════════════════════════════════════════
        # РАСЧЕТ IMPERMANENT LOSS / GAIN
        # ════════════════════════════════════════════════════════
        
        # Стратегия 1: HOLD (пассивное держание)
        # Оцениваем начальные резервы по финальной цене
        V_hold = price_after * self.x0 + self.y0
        
        # Стратегия 2: DEPOSIT (активное LP)
        # Наши текущие резервы по финальной цене
        V_deposit = price_after * self.x + self.y
        
        # Абсолютный IL (формула 11 из статьи)
        IL_absolute = V_deposit - V_hold
        
        # Относительный IL (формула 12 из статьи)
        IL_relative = (IL_absolute / V_hold) * 100 if V_hold != 0 else 0
        
        return {
            'dy': dy,
            'fee_rate': fee_rate,
            'fee_amount': dx * fee_rate,
            'gamma': gamma,
            'price_before': price_before,
            'price_after': price_after,
            'slippage': slippage,
            'effective_price': dy / dx if dx != 0 else 0,
            'K_before': K_before,
            'K_after': K_after,
            'K_growth': (K_after - K_before) / K_before * 100 if K_before != 0 else 0,
            
            # IL/IG метрики
            'V_hold': V_hold,
            'V_deposit': V_deposit,
            'IL_absolute': IL_absolute,
            'IL_relative': IL_relative,
            'IL_type': 'GAIN' if IL_absolute > 0 else 'LOSS',
            
            # Дополнительно
            'alpha': dx / self.x0  # Относительный размер обмена
        }
    
    def copy(self):
        """Создать копию пула"""
        new_pool = UniswapPool(self.x0, self.y0, self.fee_function)
        new_pool.x = self.x
        new_pool.y = self.y
        new_pool.K = self.K
        return new_pool

In [4]:
pool = UniswapPool(x0=100, y0=200_000)

fig = go.Figure()

state = pool.get_current_state()

fig.add_trace(go.Scatter(x=[state['x']], y=[state['y']], mode='markers',
    marker=dict(color='black', size=12), name=f"Начальное состояние ({state['x']}, {state['y']:,})"))

x_range = np.linspace(5, 200, 1000)
y_curve = pool.K / x_range
fig.add_trace(go.Scatter(x=x_range, y=y_curve, mode='lines', name=f'x·y = {pool.K:,}'))

'''
        - dy: Amount of Y tokens received
        - fee_rate: Fee rate applied to the swap
        - fee_amount: Fee amount (in X tokens)
        - gamma: Multiplier after fee (1 - φ)
        - price_before: Price before the swap
        - price_after: Price after the swap
        - slippage: Price slippage (in %)
'''
res = pool.swap(dx=50)
print(f"Amount of Y tokens received: {res['dy']:.1f} USDC")
print(f"Fee rate applied to the swap: {res['fee_rate']*100:.2f}%")
print(f"Fee amount (in X tokens): {res['fee_amount']:.3f} ETH")
print(f"Price before the swap: {res['price_before']:.1f} USDC/ETH")
print(f"Price after the swap: {res['price_after']:.1f} USDC/ETH")
print(f"Price slippage: {res['slippage']:.2f}%")
print(f"Effective price: {res['effective_price']:.1f} USDC/ETH")
state = pool.get_current_state()

fig.add_trace(go.Scatter(x=[state['x']], y=[state['y']], mode='markers',
    marker=dict(color='black', size=12), name=f"Состояние после swap ({state['x']}, {state['y']:,})"))

x_range = np.linspace(5, 200, 1000)
y_curve = pool.K / x_range
fig.add_trace(go.Scatter(x=x_range, y=y_curve, mode='lines', name=f'x·y = {pool.K:,}'))

fig.show()

Amount of Y tokens received: 66533.2 USDC
Fee rate applied to the swap: 0.30%
Fee amount (in X tokens): 0.150 ETH
Price before the swap: 2000.0 USDC/ETH
Price after the swap: 889.8 USDC/ETH
Price slippage: 33.47%
Effective price: 1330.7 USDC/ETH


In [5]:
def calculate_alpha_max(phi1: float, phi2: float = 0) -> float:
    """
    Вычислить границу зоны Impermanent Gain
    
    Формула из статьи:
    α_max = (1 - γ1·γ2) / [γ1·(2·γ2 - 1)]
    
    При малых комиссиях: α_max ≈ φ1 + φ2
    
    Args:
        phi1: Комиссия на отправляемый токен (обычно 0.003)
        phi2: Комиссия на получаемый токен (обычно 0)
    
    Returns:
        α_max: Граница зоны IG
    """
    gamma1 = 1 - phi1
    gamma2 = 1 - phi2
    
    # Точная формула
    if 2 * gamma2 - 1 != 0:
        alpha_max_exact = (1 - gamma1 * gamma2) / (gamma1 * (2 * gamma2 - 1))
    else:
        alpha_max_exact = float('inf')
    
    # Приближение для малых phi
    alpha_max_approx = phi1 + phi2
    
    return {
        'alpha_max_exact': alpha_max_exact,
        'alpha_max_approx': alpha_max_approx,
        'phi1': phi1,
        'phi2': phi2
    }

In [6]:
def analyze_il_vs_trade_size(
    pool: UniswapPool,
    x0: float,
    alpha_range: np.ndarray = None
) -> Dict:
    """
    Анализ IL/IG в зависимости от размера обмена
    
    Args:
        pool: Пул Uniswap
        x0: Начальный резерв X
        alpha_range: Диапазон α = dx/x0 для анализа
    
    Returns:
        Dict с результатами
    """
    if alpha_range is None:
        # От 0.01% до 2% от пула
        alpha_range = np.linspace(0.0001, 0.02, 200)
    
    results = {
        'alpha': [],
        'dx': [],
        'IL_absolute': [],
        'IL_relative': [],
        'IL_type': [],
        'dy': [],
        'fee_amount': [],
        'K_growth': [],
        'V_hold': [],
        'V_deposit': []
    }
    
    for alpha in alpha_range:
        pool.reset()
        dx = alpha * x0
        
        res = pool.swap(dx)
        
        results['alpha'].append(alpha)
        results['dx'].append(dx)
        results['IL_absolute'].append(res['IL_absolute'])
        results['IL_relative'].append(res['IL_relative'])
        results['IL_type'].append(res['IL_type'])
        results['dy'].append(res['dy'])
        results['fee_amount'].append(res['fee_amount'])
        results['K_growth'].append(res['K_growth'])
        results['V_hold'].append(res['V_hold'])
        results['V_deposit'].append(res['V_deposit'])
    
    return results

In [7]:
def visualize_il_analysis(results: Dict, alpha_max_info: Dict):
    """
    Визуализация результатов анализа IL/IG
    
    Args:
        results: Результаты analyze_il_vs_trade_size
        alpha_max_info: Информация о границе IG
    """
    alpha = np.array(results['alpha']) * 100  # В процентах
    IL_rel = np.array(results['IL_relative'])
    IL_abs = np.array(results['IL_absolute'])
    
    alpha_max = alpha_max_info['alpha_max_exact'] * 100
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(
            'Относительный IL (%)',
            'Абсолютный IL (USDC)',
            'V_hold vs V_deposit',
            'Рост K и Комиссии'
        ),
    )
    
    # 1. Относительный IL
    
    # Разделяем на GAIN и LOSS
    gain_mask = IL_rel > 0
    loss_mask = IL_rel <= 0
    
    if np.any(gain_mask):
        fig.add_trace(go.Scatter(
            x=alpha[gain_mask],
            y=IL_rel[gain_mask],
            mode='lines',
            name='Impermanent GAIN',
            line=dict(color='green', width=3),
            fill='tozeroy',
            fillcolor='rgba(0,255,0,0.2)'
        ), row=1, col=1)
    
    if np.any(loss_mask):
        fig.add_trace(go.Scatter(
            x=alpha[loss_mask],
            y=IL_rel[loss_mask],
            mode='lines',
            name='Impermanent LOSS',
            line=dict(color='red', width=3),
            fill='tozeroy',
            fillcolor='rgba(255,0,0,0.2)'
        ), row=1, col=1)
    
    # Нулевая линия
    fig.add_hline(y=0, line_dash='dash', line_color='black', row=1, col=1)
    
    # Граница α_max
    fig.add_vline(
        x=alpha_max,
        line_dash='dot',
        line_color='blue',
        annotation_text=f'α_max ≈ {alpha_max:.3f}%',
        annotation_position='right bottom',
        row=1, col=1
    )
    
    # 2. Абсолютный IL
    
    fig.add_trace(go.Scatter(
        x=alpha,
        y=IL_abs,
        mode='lines',
        name='IL (абсолютный)',
        line=dict(color='purple', width=2.5)
    ), row=1, col=2)
    
    fig.add_hline(y=0, line_dash='dash', line_color='black', row=1, col=2)
    
    # Обновляем оси
    fig.update_xaxes(title_text='α - размер обмена (% от пула)', row=1, col=1)
    fig.update_xaxes(title_text='α - размер обмена (% от пула)', row=1, col=2)
    
    fig.update_yaxes(title_text='IL относительный (%)', row=1, col=1)
    fig.update_yaxes(title_text='IL абсолютный (USDC)', row=1, col=2)
    
    fig.update_layout(
        title_text='<b>Анализ Impermanent Loss vs Impermanent Gain</b>',
        title_x=0.5,
        width=1400,
        template='plotly_white',
        showlegend=True
    )
    
    return fig

In [8]:
# Параметры пула
X0 = 100
Y0 = 200_000
    
# Фиксированная комиссия 0.3%
phi1 = 0.003
phi2 = 0.0
    
# Создаем пул
pool = UniswapPool(X0, Y0, lambda dx, x, y: phi1)

# Вычисляем границу
alpha_max_info = calculate_alpha_max(phi1, phi2)

# Анализируем диапазон от 0.01% до 0.5%
alpha_range = np.linspace(0.0001, 0.005, 300)
results = analyze_il_vs_trade_size(pool, X0, alpha_range)

fig = visualize_il_analysis(results, alpha_max_info)
fig.show()

## Устойчивость константной комиссии к дроблению

**Вопрос:** выгодно ли трейдеру разбить один большой обмен $\Delta x$ на $n$ маленьких $\Delta x / n$?

Комиссия в Uniswap v2 работает так: при обмене $\Delta x$ в пул попадает только $\gamma \cdot \Delta x$ (где $\gamma = 1 - \varphi$), а оставшаяся часть $\varphi \cdot \Delta x$ — комиссия, которая **реинвестируется** в пул. Из-за этого после каждого обмена инвариант $K$ растёт.

### Как мы измеряем выгоду

$$\text{advantage} = \frac{\Delta y_{\text{split}}^{(n)} - \Delta y_{\text{single}}}{\Delta y_{\text{single}}} \cdot 100\%$$

где:
- $\Delta y_{\text{single}}$ — сколько USDC получим за один обмен $\Delta x$
- $\Delta y_{\text{split}}^{(n)}$ — сколько USDC получим суммарно, если разобьём $\Delta x$ на $n$ равных обменов

Если `advantage > 0` — дробить выгодно (комиссия неустойчива).  
Если `advantage ≈ 0` — дробление нейтрально (комиссия устойчива).

### Дополнительные метрики

- **$K_n / K_1$** — отношение инварианта после $n$ обменов к инварианту после одного. Показывает, сколько комиссий собрал пул в каждом случае. Если $K_n / K_1 > 1$, LP заработали больше при дроблении.
- **$\Delta y_{\text{total}} - \Delta y_{\text{single}}$** — абсолютная разница в USDC

In [18]:
X0 = 100
Y0 = 200_000
PHI = 0.003  # 0.3%

dx_values = [0.5, 1, 2, 5, 10, 20]
max_n = 50

pool = UniswapPool(X0, Y0)

# Для каждого размера обмена: сравниваем 1 большой vs n маленьких
all_results = {}

for dx_total in dx_values:
    # Базовый случай: один обмен
    pool.reset()
    res_single = pool.swap(dx_total)
    dy_single = res_single['dy']
    K_single = pool.K

    n_list = []
    advantage_list = []
    dy_list = []
    K_ratio_list = []

    for n in range(1, max_n + 1):
        pool.reset()
        dx_part = dx_total / n
        dy_total = 0

        for _ in range(n):
            res = pool.swap(dx_part)
            dy_total += res['dy']

        K_n = pool.K
        advantage = (dy_total - dy_single) / dy_single * 100
        
        n_list.append(n)
        advantage_list.append(advantage)
        dy_list.append(dy_total)
        K_ratio_list.append(K_n / K_single)

    all_results[dx_total] = {
        'n': n_list,
        'advantage': advantage_list,
        'dy': dy_list,
        'K_ratio': K_ratio_list,
        'dy_single': dy_single
    }

# Печатаем сводку
print(f"Пул: {X0} ETH × {Y0:,} USDC, комиссия: {PHI*100:.2f}%")
print()
print(f"{'dx (ETH)':<12} {'dx/X0':<10} {'Δy (n=1)':<16} {'Выгода n=10':<16} {'Выгода n=50':<16}")
print("-" * 70)

for dx_total in dx_values:
    r = all_results[dx_total]
    print(f"{dx_total:<12} {dx_total/X0*100:.1f}%{'':5} {r['dy_single']:>12,.2f}     {r['advantage'][9]:>+.6f}%     {r['advantage'][49]:>+.6f}%")

Пул: 100 ETH × 200,000 USDC, комиссия: 0.30%

dx (ETH)     dx/X0      Δy (n=1)         Выгода n=10      Выгода n=50     
----------------------------------------------------------------------
0.5          0.5%            992.05     -0.000670%     -0.000730%
1            1.0%          1,974.32     -0.001332%     -0.001451%
2            2.0%          3,910.03     -0.002628%     -0.002863%
5            5.0%          9,496.59     -0.006314%     -0.006885%
10           10.0%         18,132.22     -0.011846%     -0.012933%
20           20.0%         33,249.96     -0.021006%     -0.022991%


In [12]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Выгода от дробления (%)',
        'Отношение инвариантов K_n / K_1',
        'Абсолютная разница Δy (USDC)',
        'Получено USDC'
    )
)

colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3']

for i, dx_total in enumerate(dx_values):
    r = all_results[dx_total]
    color = colors[i]
    label = f'{dx_total} ETH ({dx_total/X0*100:.1f}%)'

    # 1. Выгода от дробления
    fig.add_trace(go.Scatter(
        x=r['n'], y=r['advantage'], mode='lines+markers',
        name=label, line=dict(color=color, width=2), marker=dict(size=3),
        legendgroup=label, showlegend=True
    ), row=1, col=1)

    # 2. K_n / K_1
    fig.add_trace(go.Scatter(
        x=r['n'], y=r['K_ratio'], mode='lines+markers',
        name=label, line=dict(color=color, width=2), marker=dict(size=3),
        legendgroup=label, showlegend=False
    ), row=1, col=2)

    # 3. Абсолютная разница
    dy_diff = [dy - r['dy_single'] for dy in r['dy']]
    fig.add_trace(go.Scatter(
        x=r['n'], y=dy_diff, mode='lines+markers',
        name=label, line=dict(color=color, width=2), marker=dict(size=3),
        legendgroup=label, showlegend=False
    ), row=2, col=1)

    # 4. Получено USDC
    fig.add_trace(go.Scatter(
        x=r['n'], y=r['dy'], mode='lines',
        name=label, line=dict(color=color, width=2),
        legendgroup=label, showlegend=False
    ), row=2, col=2)

# Нулевые линии
fig.add_hline(y=0, line_dash='dash', line_color='black', opacity=0.5, row=1, col=1)
fig.add_hline(y=1, line_dash='dash', line_color='black', opacity=0.5, row=1, col=2)
fig.add_hline(y=0, line_dash='dash', line_color='black', opacity=0.5, row=2, col=1)

fig.update_xaxes(title_text='Количество обменов (n)', row=1, col=1)
fig.update_xaxes(title_text='Количество обменов (n)', row=1, col=2)
fig.update_xaxes(title_text='Количество обменов (n)', row=2, col=1)
fig.update_xaxes(title_text='Количество обменов (n)', row=2, col=2)

fig.update_yaxes(title_text='Выгода (%)', row=1, col=1)
fig.update_yaxes(title_text='K_n / K_1', row=1, col=2)
fig.update_yaxes(title_text='Δy_total - Δy_single (USDC)', row=2, col=1)
fig.update_yaxes(title_text='Получено USDC', row=2, col=2)

fig.update_layout(
    title_text='Устойчивость константной комиссии к дроблению',
    width=1200, height=800,
    template='plotly_white'
)

fig.show()

## Вывод

Advantage **отрицательный** для всех размеров обмена и любого $n$. Это значит, что дробление **невыгодно** трейдеру — он всегда получает меньше USDC, чем при одном большом обмене.

# Попробуем проанализировать устойчивость к дроблению

$X$, $Y$ --- резервы токенов в пуле с инвариантом:
\begin{equation}
X \cdot Y = K
\end{equation}

Трейдер желает обменять $\Delta x_{\text{total}}$ на $Y$ токены.

## Две стратегии

1. **Один большой своп:**

$$\Delta x_{\text{total}} \to \Delta y_{\text{single}}$$

2. **$n$ мелких свопов:**

$$\frac{\Delta x_{\text{total}}}{n} \cdot n \to \Delta y_{\text{total}}$$

### Задача

> **Цель:** Найти комиссию $\phi = \phi(\alpha)$, где $\alpha = \frac{dx}{x}$, такую что:
>
> $$\Delta y_{\text{total}}(n) = \Delta y_{\text{single}} \quad \forall\, n \in \mathbb{N}$$

## 2. Один большой обмен

### Дано

- Начальное состояние пула: $(x_0, y_0)$
- Размер обмена: $\Delta x_{\text{total}}$
- Относительный размер: $\alpha_0 = \dfrac{\Delta x_{\text{total}}}{x_0}$

### Вход

Трейдер отправляет $\Delta x_{\text{total}}$ в пул.

### Комиссия

Взимается комиссия $\phi(\alpha_0) \cdot \Delta x_{\text{total}}$.

В пул попадает часть:

$$\gamma(\alpha_0) \cdot \Delta x_{\text{total}} = [1 - \phi(\alpha_0)] \cdot \Delta x_{\text{total}}$$

### Состояние после обмена

После обмена - часть, которая формирует объем отдачи y:

$$x_1 = x_0 + \gamma(\alpha_0) \cdot \Delta x_{\text{total}} = x_0[1 + \gamma(\alpha_0) \cdot \alpha_0]$$

Полный резерв:

$$X = x_0 + \Delta x_{\text{total}}$$

## 3. $n$ последовательных обменов

### Разбиваем $\Delta x_{\text{total}}$ на $n$ частей

Размер каждого обмена:

$$\Delta x_i = \frac{\Delta x_{\text{total}}}{n}$$

### Обмен 1

**Входной резерв:** $x_0$

**Размер обмена:** $\Delta x_1 = \dfrac{\Delta x_{\text{total}}}{n}$

**Относительный размер:**

$$\alpha_1 = \frac{\Delta x_1}{x_0} = \frac{1}{n} \cdot \frac{\Delta x_{\text{total}}}{x_0} = \frac{\alpha_0}{n}$$

**В пул попадает:**

$$\gamma(\alpha_1) \cdot \Delta x_1 = \gamma\left(\frac{\alpha_0}{n}\right) \cdot \frac{\alpha_0}{n} \cdot x_0$$

**Новый виртуальный резерв:**

$$x_1 = x_0 + x_0 \cdot \gamma\left(\frac{\alpha_0}{n}\right) \cdot \frac{\alpha_0}{n}$$

**Физический резерв:** $X = x_0 + \Delta x_1$

### Обмен 2

**Входной резерв:** $x_0 + \Delta x_1$

**Размер обмена:** $\Delta x_1$

**Относительный размер:**

$$\alpha_2 = \frac{\Delta x_1}{x_0 + \Delta x_1} = \frac{\alpha_0/n}{1 + \alpha_0/n} = \frac{\alpha_0}{n + \alpha_0}$$

### Обмен 3

Аналогично:

$$\alpha_3 = \frac{\alpha_0/n}{1 + 2\alpha_0/n} = \frac{\alpha_0}{n + 2\alpha_0}$$

### Общая формула

**Обмен $i$:**

**Входной резерв:** $x_0 + (i-1) \cdot \Delta x$

**Относительный размер:**

$$\alpha_i = \frac{\alpha_0}{n + (i-1) \cdot \alpha_0}$$

где $\alpha_0 = \dfrac{\Delta x_{\text{total}}}{x_0}$.

**В пул попадает:** $\gamma(\alpha_i) \cdot \Delta x$

**Физический резерв после:** $x_0 + i \cdot \Delta x$

## 4. Полученное количество $Y$ через мгновенную цену

### Мгновенная цена

Из инварианта $x \cdot y = K$ следует $y = K/x$, мгновенная цена:

$$p(x, K) = -\frac{dy}{dx} = \frac{K}{x^2}$$

### Полученное $Y$

$$\Delta y = \int_{x_0}^{x_f} \frac{K_0}{x^2}\,dx = K_0 \left(\frac{1}{x_0} - \frac{1}{x_f}\right)$$

### Один обмен

$$\Delta y_{\text{single}} = K_0 \left(\frac{1}{x_0} - \frac{1}{x_0 + \gamma(\alpha_0) \cdot \Delta x_{\text{total}}}\right)$$

### $n$ обменов

Для $i$-го обмена:

$$\Delta y_i = K_{i-1} \left(\frac{1}{x_0 + (i-1)\Delta x} - \frac{1}{x_0 + (i-1)\Delta x + \gamma(\alpha_i) \cdot \Delta x}\right)$$

где $K_i$ обновляется после каждого обмена (растёт из-за реинвестирования комиссии).

## 5. Условие устойчивости

> **Условие устойчивости:**
>
> $$\Delta y_{\text{single}} = \sum_{i=1}^{n} \Delta y_i = \Delta y_{\text{total}}$$
>
> где:
>
> $$\Delta y_{\text{single}} = K_0 \left(\frac{1}{x_0} - \frac{1}{x_0 + \gamma(\alpha_0) \cdot \Delta x_{\text{total}}}\right)$$
>
> $$\Delta y_i = K_{i-1} \left(\frac{1}{x_0 + (i-1)\Delta x} - \frac{1}{x_0 + (i-1)\Delta x + \gamma(\alpha_i) \cdot \Delta x}\right)$$
